## Assignment 1

We first import the necessary libraries:

In [101]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
from numba import njit, float64
from numba.experimental import jitclass
from scipy.optimize import fsolve
from sympy.solvers import solve
import numpy as np

Now, we define an instance of the optimal growth model. 

In [110]:
class ogm():

    def __init__(self, σ=1, β=0.984, δ=0.025, α=1/3, k0=1):

        self.σ, self.β, self.δ, self.α, self.k0 = σ, β, δ, α, k0

    def u(self, c):
        '''
        Utility function
        ASIDE: If you have a utility function that is hard to solve by hand
        you can use automatic or symbolic differentiation
        See https://github.com/HIPS/autograd
        '''
        σ = self.σ

        return c ** (1 - σ) / (1 - σ) if σ!= 1 else np.log(c)

    def u_prime(self, c):
        'Derivative of utility'
        σ = self.σ

        return c ** (-σ)

    def u_prime_inv(self, c):
        'Inverse of derivative of utility'
        σ = self.σ

        return c ** (-1 / σ)

    def f(self, k):
        'Production function'
        α = self.α

        return k ** α

    def f_prime(self, k):
        'Derivative of production function'
        α = self.α

        return α * k ** (α - 1)

    def f_prime_inv(self, k):
        'Inverse of derivative of production function'
        α = self.α 

        return (k / α) ** (1 / (α - 1))
    
    def ss(self, k):
        'Define the steady-state equation'
        β, δ = self.β, self.δ
        f_prime = self.f_prime
        
        return β*(f_prime(k) + (1-δ)) - 1
        
    def find_ss(self):
        'Solve the steady-state equation'
        ss = self.ss
        steady_state = fsolve(ss, 1e-10)
        return steady_state[0]

    def next_k_c(self, k, c):
        ''''
        Given the current capital Kt and an arbitrary feasible
        consumption choice Ct, computes Kt+1 by state transition law
        and optimal Ct+1 by Euler equation.
        '''
        β, δ = self.β, self.δ
        u_prime, u_prime_inv = self.u_prime, self.u_prime_inv
        f, f_prime = self.f, self.f_prime

        k_next = f(k) + (1 - δ) * k - c
        c_next = u_prime_inv(u_prime(c) / (β * (f_prime(k_next) + (1 - δ))))

        return k_next, c_next  
           
        

In [113]:
## Todo: Make endogenous grid inside class

# find out where to initialize that